<a href="https://colab.research.google.com/github/matthew-boccio/test/blob/master/Macroeconomic_Analysis_Methodology_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Macro Economic Analysis: Basic Methodology

Thank you for looking at my code for this analysis! I hope that it helps anyone interested look at a number of different macroeconomic variables in relation to their conversion events and see what the effect of this weird economy is on your conversion metrics. 

In [ ]:
#import relevant packages 

import numpy as np
import pandas as pd
from sklearn import linear_model
import math
import seaborn as sns


from matplotlib import pyplot as plt
plt.style.use('ggplot')
!pip install fredapi
# uncomment the pip install if you don't already have fredapi installed as a package 
from fredapi import Fred

Import Kepler Data from Datorama - please note sample file template!

We recommend keeping things simple to start! Spend and Conversions by Month works best 

Make sure that you have Month as the first column, Spend as the second column, and Conversions as the third column!

Make sure your file is either a .xlsx file or a .csv file! 

Works best with Kepler data that matches the Macro Economic data. So 1/1/18 - the previous month is what I'd do

In [ ]:
#DON'T SKIP: replace "Kepler_Data_From_Datorama" with your file name!

try: 
    kepler_data_frame = pd.read_excel("Kepler_Data_From_Datorama.xlsx")
except FileNotFoundError:
    kepler_data_frame = pd.read_csv("Kepler_Data_From_Datorama.csv")

#DON'T SKIP: put in the start and end date of your file 
#input the first month for your file: year, then month, then day (1) 

start_date = "2018-06-01" #@param {type : "date"}
end_date = "2022-10-01" #@param {type : "date"}

kepler_data_frame = kepler_data_frame.rename(columns = {"index": "Month", kepler_data_frame.columns[1]:"Total_Spend", kepler_data_frame.columns[2]: "Conversions"})

#if your BU did not pause for Covid, skip this step: 
# you can skip by highlighting everything between 'start skip' and 'stop skip' and pressing "ctrl + /"

# the spend and converison values in this data frame are arbitrary
#if you choose to use the fill_in_data_frame, please input the values that make the most sense for your business unit 

#start skip
fill_in_data_frame = pd.DataFrame({"Month": ["Apr 2020", "May 2020", "Jun 2020", "Jul 2020"], 
                     "Total_Spend": ["$900,000.00", "$800,000.00", "$750,000.00", "$700,000.00"],
                     "Conversions": ["400", "350", "200", "250"]})

fill_in_data_frame

kepler_data_frame = pd.concat([kepler_data_frame, fill_in_data_frame]).reset_index().drop("index", axis = 1)
#stop skip

kepler_data_frame

In [ ]:
#convert to timestamp object and sort by date 

for i in list(range(len(kepler_data_frame["Month"]))):
    kepler_data_frame["Month"][i] = pd.Timestamp(kepler_data_frame["Month"][i])
    
kepler_data_frame = kepler_data_frame.sort_values("Month").reset_index().drop("index", axis = 1)

kepler_data_frame = kepler_data_frame[(kepler_data_frame["Month"] >= start_date) & (kepler_data_frame["Month"] <= end_date)].reset_index().drop("index", axis = 1)


kepler_data_frame

To build your data frame with accurate and up to date macroeconomic data, a connection to the St. Louis Federal Reserve (FRED) is necessary. Register an account with FRED, then get your own personal API key and insert it into the first line of code: 

1. Go to https://fred.stlouisfed.org/, click "My Account", click "Create New Account", and register with your Kepler Email 
2. Once you have an account, go to "My Account" and click "API Keys". In this window, click "Request API Key" 
3. Describe your application briefly in the text box and request the key. The key you request should appear shortly after you request it. Once you have your API key, paste it into the cell below! Replace "API KEY" with your own API Key 

In [ ]:
#please input your own API key after 'api_key = '
fred = Fred(api_key='957a41fad909ac1dbd95403a94269991')

#below, 'data' is an example series that you can pull from the FRED API. This series is inflation rate 
data = fred.get_series('CPALTT01USM659N', observation_start = "1/1/1968")
data

In [ ]:
#we're going to make a data frame with a few different macroeconomic variables, listed below: 

macro_data_frame = pd.DataFrame([fred.get_series("UNRATE", observation_start = "1/1/1968"), #Unemployment (UNRATE) 
                                fred.get_series("CPALTT01USM659N", observation_start = "1/1/1968"), #Inflation(CPALTT01USM659N) - round this to one digit after decimal 
                                fred.get_series("JHDUSRGDPBR", observation_start = "1/1/1968"), #Recession(JHDUSRGDPBR) - binary, 1 = recession, 0 = no recession 
                                fred.get_series("FEDFUNDS", observation_start = "1/1/1968"), #Fed Interest Rate(FEDFUNDS) 
                                fred.get_series("GDPC1", observation_start = "1/1/1968")])#Real GDP (GDPC1)

#for more macroeconomic variables, find them using fred.search(your variable), or go to fred.stlouisfed.org

macro_data_frame = macro_data_frame.transpose().reset_index()
macro_data_frame = macro_data_frame.rename(columns = {"index": "Date", 0:"Unemployment_Rate", 1:"Inflation_Pct", 2:"Recession_Dummy", 3:"Fed_Interest_Rate", 4:"Real_GDP"})
macro_data_frame

In [ ]:
#get rid of N/A values 
print(macro_data_frame.isna().sum())
macro_data_frame = macro_data_frame.fillna(0)

#round inflation percentages to single digit past the decimal 
for i in list(range(len(macro_data_frame["Inflation_Pct"]))):
    macro_data_frame["Inflation_Pct"][i] = round(macro_data_frame["Inflation_Pct"][i], 1)

#this smooths quarterly GDP into monthly
for i in list(range(len(macro_data_frame["Real_GDP"]))): 
    if macro_data_frame["Real_GDP"][i] == 0 and i > (len(macro_data_frame["Real_GDP"])-4):
        macro_data_frame["Real_GDP"][i] = macro_data_frame["Real_GDP"][i-1]
    elif macro_data_frame["Real_GDP"][i] == 0 and i%3 == 1 and i <= len(macro_data_frame["Real_GDP"])-2 and macro_data_frame["Real_GDP"][i+2] != 0:
        macro_data_frame["Real_GDP"][i] = ((macro_data_frame["Real_GDP"][i+2] - macro_data_frame["Real_GDP"][i-1])/3) + macro_data_frame["Real_GDP"][i-1]
    elif macro_data_frame["Real_GDP"][i] == 0 and i%3 == 2 and i <= len(macro_data_frame["Real_GDP"])-1 and macro_data_frame["Real_GDP"][i+1] != 0:
        macro_data_frame["Real_GDP"][i] = 2 * ((macro_data_frame["Real_GDP"][i+1] - macro_data_frame["Real_GDP"][i-2])/3) + macro_data_frame["Real_GDP"][i-2]

#making a relative change in GDP line, so we can see inc/dec        
macro_data_frame["Real_GDP_%_Change"] = 0.0
for i in list(range(len(macro_data_frame["Real_GDP"]))): 
    if i > 0 and macro_data_frame["Real_GDP"][i] != 0:
        macro_data_frame["Real_GDP_%_Change"][i] = round(((macro_data_frame["Real_GDP"][i]/macro_data_frame["Real_GDP"][i-1])-1)*100, 2)
    elif i > 0 and macro_data_frame["Real_GDP"][i] == 0:
        macro_data_frame["Real_GDP_%_Change"][i] = 0
        
#for some reason, inflation didn't fill in for Aug 2022 and Sept 2022. 
#We have those numbers from reports, so we'll fill it in manually 
macro_data_frame["Inflation_Pct"][655] = 8.3
macro_data_frame["Inflation_Pct"][656] = 8.2
    
macro_data_frame

In [ ]:
#let's do some EDA! Here's some general summary statistics about our macroeconomic variables 

macro_data_frame.describe()

In [ ]:
#here's a summary histogram outlining inflation percentages

plt.hist(macro_data_frame["Inflation_Pct"], bins = 50)

In [ ]:
#here's a summary histogram outlining unemployment percentages

plt.hist(macro_data_frame["Unemployment_Rate"], bins = 50)

In [ ]:
#a quick box plot outlining average unemployment during a recession period and a non-recession period 

fig = plt.figure(figsize=(15, 9))
sns.boxplot(y=macro_data_frame["Unemployment_Rate"], x=macro_data_frame["Recession_Dummy"])
plt.show()

In [ ]:
#a quick box plot outlining average inflation during a recession period and a non-recession period 


fig = plt.figure(figsize=(15, 9))
sns.boxplot(y=macro_data_frame["Inflation_Pct"], x=macro_data_frame["Recession_Dummy"])
plt.show()

In [ ]:
macro_data_frame.corr()

In [ ]:
# Basic correlogram
sns.pairplot(macro_data_frame)
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(8,4))

macro_data_frame[["Date", "Inflation_Pct", "Recession_Dummy"]].plot(ax=axs, x = "Date", y = "Inflation_Pct", grid = True)
macro_data_frame[["Date", "Inflation_Pct", "Recession_Dummy"]].plot(ax=axs, x = "Date", y = "Recession_Dummy", secondary_y = True)

plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(8,4))

macro_data_frame[["Date", "Inflation_Pct", "Real_GDP_%_Change"]].plot(ax=axs, x = "Date", y = "Inflation_Pct", grid = True)
macro_data_frame[["Date", "Inflation_Pct", "Real_GDP_%_Change"]].plot(ax=axs, x = "Date", y = "Real_GDP_%_Change", secondary_y = True)

plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(8,4))

macro_data_frame[["Date", "Real_GDP_%_Change"]].plot(ax=axs, x = "Date", y = "Real_GDP_%_Change", grid = True)


plt.show()

In [ ]:
macro_data_frame_1 = macro_data_frame[(macro_data_frame["Date"] >= start_date) & (macro_data_frame["Date"] <= end_date)].reset_index().drop("index", axis = 1)

macro_data_frame_1

In [ ]:
#combine data frames! Make sure that all the data matches 

regression_data_frame = pd.concat([macro_data_frame_1, kepler_data_frame], axis = 1).drop(["Month"], axis = 1)
regression_data_frame

In [ ]:
#add seasonality 

regression_data_frame["Month_No"] = 0
for i in list(range(len(regression_data_frame["Date"]))):
    regression_data_frame["Month_No"][i] = regression_data_frame["Date"][i].month
regression_data_frame["Season"] = "0"
for i in list(range(len(regression_data_frame["Month_No"]))):
    if regression_data_frame["Month_No"][i] == 12 or regression_data_frame["Month_No"][i] < 3:
        regression_data_frame["Season"][i] = "Winter"
    elif regression_data_frame["Month_No"][i] >= 3 and regression_data_frame["Month_No"][i] < 6:
        regression_data_frame["Season"][i] = "Spring"
    elif regression_data_frame["Month_No"][i] >= 6 and regression_data_frame["Month_No"][i] < 9:
        regression_data_frame["Season"][i] = "Summer"
    elif regression_data_frame["Month_No"][i] >= 9 and regression_data_frame["Month_No"][i] < 12:
        regression_data_frame["Season"][i] = "Fall"
        
seasons_dummy = regression_data_frame["Season"]
seasons_dummy = pd.get_dummies(seasons_dummy).drop(["Fall"], axis = 1)

regression_data_frame = pd.concat([regression_data_frame, seasons_dummy], axis = 1, join = "outer")
regression_data_frame = regression_data_frame.drop(["Month_No", "Season"], axis = 1)   

regression_data_frame

In [ ]:
#data cleaning, making sure that Spend and Conversions are regression ready 

import re

try:
    for i in list(range(len(regression_data_frame["Total_Spend"]))):
        regression_data_frame["Total_Spend"][i] = (np.float64(re.sub("\D", "", regression_data_frame["Total_Spend"][i])))/100
except TypeError: 
    print("Spend is of the Correct Type")
try:
    for i in list(range(len(regression_data_frame["Conversions"]))):
        regression_data_frame["Conversions"][i] = np.float64(re.sub("\D", "", regression_data_frame["Conversions"][i]))
except TypeError:
    print("Conversions are of the Correct Type")

data_types_dict = {'Total_Spend': np.float64, "Conversions": np.float64}
  
regression_data_frame = regression_data_frame.astype(data_types_dict)


regression_data_frame

In [ ]:
#data cleaning, breaking out spend by thousands of dollars. if your team has lower spend, comment out this step 

regression_data_frame["Total_Spend (Thousands)"] = np.float64(0)

for i in list(range(len(regression_data_frame["Total_Spend (Thousands)"]))):
    regression_data_frame["Total_Spend (Thousands)"][i] = (regression_data_frame["Total_Spend"][i])/1000

regression_data_frame = regression_data_frame.drop(["Total_Spend"], axis = 1)

regression_data_frame

In [ ]:
#regression time! Importing packages here 

from sklearn import linear_model
ols = linear_model.LinearRegression()

## separate the input variables and output variables - add and remove columns in X as you see fit!
X = regression_data_frame.drop(["Conversions", "Date",  "Real_GDP", "Real_GDP_%_Change", "Fed_Interest_Rate",  "Recession_Dummy"], axis = 1) # input variable
Y = regression_data_frame['Conversions']  # output variable
ols.fit(X, Y)
print("Intercept: %f" %ols.intercept_)
print("Coefficients: %s" %str(ols.coef_))
print("R^2: %f" %(ols.score(X, Y)))

In [ ]:
#output and coefficients 

import statsmodels.api as sm 
X_add_const = sm.add_constant(X)
output_1 = sm.OLS(Y, X_add_const)
ans = output_1.fit()
print(ans.summary())

In [ ]:
from sklearn import linear_model
ols2 = linear_model.LinearRegression()
x = regression_data_frame["Total_Spend (Thousands)"] # input varialbe
y = regression_data_frame['Conversions']  # output variable

plt.figure(figsize=(9, 6))
plt.scatter(x, y, c='b')
plt.xlabel('Spend')
plt.ylabel('GCT')
plt.show()

In [ ]:
sns.pairplot(regression_data_frame)
plt.show()

In [ ]:
regression_data_frame.to_csv("regression_data_frame_complete.csv")